In [1]:
import pandas as pd
import panel as pn
import numpy as np
from sqlalchemy import create_engine
from datetime import date, timedelta
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set

pn.extension('tabulator')

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 2, 19)

### Set today = last closed business day

In [2]:
today = today - timedelta(days=1)
today

datetime.date(2022, 2, 18)

### Restart and Run All Cells

In [3]:
sqlUpd = """
UPDATE buy B
SET dividend =
(SELECT DIVIDEND FROM dividend D
WHERE B.name = D.name)"""
rp = const.execute(sqlUpd)
rp.rowcount

33

In [4]:
cols = 'name period shares unit_cost cost_amt base_amt pct'.split()

In [5]:
format_dict = {
    'shares':'{:,}','div_rnk':'{:,}','amt_rnk':'{:,}',  
    'price':'{:.2f}','unit_cost':'{:.2f}','mkt_price':'{:.2f}',
    'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','mkt_amt':'{:,.2f}',
    'profit':'{:,.2f}','base_amt':'{:,.2f}','cumsum':'{:,.2f}',
    'dividend':'{:.4f}',    
    'pct':'{:,.2f}%','cost_pct':'{:,.2f}%','mkt_pct':'{:,.2f}%','pft_pct':'{:,.2f}%',
}

### Change period of stock in table buy

In [6]:
name = 'SCC'
sql = """
SELECT * 
FROM buy 
WHERE name = '%s' 
"""
sql = sql % name
df = pd.read_sql(sql, const)
df

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,SCC,2021-09-20,900.0,398.0,0.0,0.0,1,9.9999,4,A2


In [7]:
grade = 'A2'
period = '4'
sqlUpd = """
UPDATE buy
SET period = '%s', grade = '%s'
WHERE name = '%s' 
"""
sqlUpd = sqlUpd % (period, grade, name)
print(sqlUpd)


UPDATE buy
SET period = '4', grade = 'A2'
WHERE name = 'SCC' 



In [8]:
rp = const.execute(sqlUpd)
rp.rowcount

1

### Discrepancies calculation

In [9]:
sql = '''
SELECT *
FROM buy
WHERE active = 1'''
df = pd.read_sql(sql, const)
df.drop(['volsell', 'volbal'], axis=1, inplace=True)
df.rename(columns={'volbuy': 'shares','price':'unit_cost'}, inplace=True)
df['shares'] = df['shares'].astype('int64')
df['cost_amt'] = df.shares * df.unit_cost
df.shape

(33, 9)

In [10]:
sql = '''
SELECT name, market
FROM stocks
ORDER BY name'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(359, 2)

In [11]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))    
]
values = [450000.0,300000.0,150000.0]

In [12]:
my_stocks["base_amt"] = np.select(filters, values, default=150000)
my_stocks.head().style.format(format_dict)

,name,market,base_amt
0,2S,SET,"150,000.00"
1,AAV,sSET / SETTHSI,"150,000.00"
2,ACE,SET100,"300,000.00"
3,ADVANC,SET50 / SETHD / SETTHSI,"450,000.00"
4,AEONTS,SET100,"300,000.00"


In [13]:
df_merge = pd.merge(df, my_stocks, on='name', how='inner')
df_merge.shape

(33, 11)

In [14]:
df_merge['pct'] = round(df_merge.cost_amt/df_merge.base_amt*100,2)
mask = df.period != '2'
df_merge[mask][cols].head().style.format(format_dict)

,name,period,shares,unit_cost,cost_amt,base_amt,pct
2,IVL,3,"4,800",42.00,"201,600.00","450,000.00",44.80%
3,KCE,1,"10,000",77.25,"772,500.00","450,000.00",171.67%
7,GLOBAL,4,"15,000",19.40,"291,000.00","450,000.00",64.67%
8,WHART,4,"10,000",12.50,"125,000.00","150,000.00",83.33%
9,BCH,3,"15,000",21.30,"319,500.00","300,000.00",106.50%


In [15]:
df_merge[mask].nlargest(5, 'pct')[cols].style.format(format_dict)

,name,period,shares,unit_cost,cost_amt,base_amt,pct
3,KCE,1,"10,000",77.25,"772,500.00","450,000.00",171.67%
26,SYNEX,3,"15,000",29.50,"442,500.00","300,000.00",147.50%
23,NER,3,"27,000",7.45,"201,150.00","150,000.00",134.10%
17,SIS,3,"4,500",40.50,"182,250.00","150,000.00",121.50%
9,BCH,3,"15,000",21.30,"319,500.00","300,000.00",106.50%


In [16]:
df_merge[mask].nsmallest(5, 'pct')[cols].style.format(format_dict)

,name,period,shares,unit_cost,cost_amt,base_amt,pct
18,CKP,3,"20,000",5.00,"100,000.00","300,000.00",33.33%
2,IVL,3,"4,800",42.00,"201,600.00","450,000.00",44.80%
10,SINGER,3,"4,500",42.25,"190,125.00","300,000.00",63.38%
24,MEGA,3,"4,500",43.00,"193,500.00","300,000.00",64.50%
7,GLOBAL,4,"15,000",19.40,"291,000.00","450,000.00",64.67%


### Rank

In [17]:
sql = '''
SELECT B.name, volbuy, B.price AS unit_cost, 
dividend, P.price AS mkt_price, period
FROM buy B 
JOIN price P
ON B.name = P.name
WHERE P.date = "%s"
AND active = 1
ORDER BY period, name'''
sql = sql % today 
print(sql)


SELECT B.name, volbuy, B.price AS unit_cost, 
dividend, P.price AS mkt_price, period
FROM buy B 
JOIN price P
ON B.name = P.name
WHERE P.date = "2022-02-18"
AND active = 1
ORDER BY period, name


In [18]:
cols = 'name period shares unit_cost cost_amt base_amt pct'.split()
colt = 'name shares unit_cost cost_amt mkt_price mkt_amt amt_rnk mkt_pct div_rnk'.split()
colu = 'name shares unit_cost cost_amt mkt_price mkt_amt dividend div_amt mkt_pct div_rnk amt_rnk'.split()

In [19]:
df = pd.read_sql(sql, const)
df.rename(columns={'volbuy': 'shares'}, inplace=True)
df['shares'] = df.shares.astype(int)
df['cost_amt'] = round(df['shares'] * df['unit_cost'], 2)
df['mkt_amt'] = round(df['shares'] * df['mkt_price'], 2)
df['div_amt'] = round(df['shares'] * df['dividend'], 2)
df['cost_pct'] = round(df['div_amt'] / df['cost_amt'] * 100,2)
df['mkt_pct'] = round(df['div_amt'] / df['mkt_amt'] * 100,2)
df['pft_pct'] = round((df['mkt_amt'] - df['cost_amt']) / df['cost_amt'] * 100,2)
df['div_rnk'] = df.mkt_pct.rank(ascending=False)
df['amt_rnk'] = df.mkt_amt.rank(ascending=False)
df[colu].sort_values(['div_rnk'],ascending=[True]).head().style.format(format_dict)

,name,shares,unit_cost,cost_amt,mkt_price,mkt_amt,dividend,div_amt,mkt_pct,div_rnk,amt_rnk
7,NOBLE,"63,000",7.00,"441,000.00",6.25,"393,750.00",0.8500,"53,550.00",13.60%,1.0,8.0
11,TMT,"27,000",10.50,"283,500.00",10.40,"280,800.00",1.3000,"35,100.00",12.50%,2.0,17.0
6,MCS,"50,000",16.70,"835,000.00",14.60,"730,000.00",1.3700,"68,500.00",9.38%,3.0,2.0
5,JASIF,"100,000",10.00,"1,000,000.00",11.10,"1,110,000.00",0.9500,"95,000.00",8.56%,4.0,1.0
18,RCL,"4,500",44.50,"200,250.00",43.75,"196,875.00",3.5000,"15,750.00",8.00%,5.0,22.0


In [20]:
df2 = df[df['period'] == '2'].copy()
df2['div_rnk'] = df2.mkt_pct.rank(ascending=False)
df2['amt_rnk'] = df2.mkt_amt.rank(ascending=False)
df2_out = df2[colt].sort_values('div_rnk')
df2_tab = pn.widgets.Tabulator(df2_out, layout='fit_data', width=900)
df2_tab

Tabulator(layout='fit_data', value=     name  shares  ..., width=900)

In [25]:
file_name = 'rank-div-market.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df2.sort_values(['div_rnk'],ascending=[True]).to_csv(output_file, index=False)
df2.sort_values(['div_rnk'],ascending=[True]).to_csv(data_file, index=False)
df.sort_values(['div_rnk'],ascending=[True]).to_csv(box_file, index=False)
df2.sort_values(['div_rnk'],ascending=[True]).to_csv(one_file, index=False)

In [ ]:
df.period.value_counts()

In [ ]:
df.period.value_counts(normalize=True).to_frame().style.format('{:.1%}')

In [ ]:
Period_Count =\
df.period.value_counts(sort=True).nlargest(3)
plt.figure(figsize=(10,5))
sns.barplot(Period_Count.index,
Period_Count.values, alpha=0.8)
plt.show()

In [ ]:
df9 = df[df['period'] != '2'].copy()
df9['div_rnk'] = df9.mkt_pct.rank(ascending=False)
df9['amt_rnk'] = df9.mkt_amt.rank(ascending=False)
df9_out = df9[colt].sort_values('div_rnk')
df9_out.set_index('name', inplace=True)
df9_tab = pn.widgets.Tabulator(df9_out, layout='fit_data', width=900)
df9_tab

In [ ]:
short_term = (df.period == '4')
df.loc[short_term].head().style.format(format_dict)

In [ ]:
data = df.loc[short_term]
names = data['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = '''
SELECT name, min_price AS min, price, max_price AS max
FROM stocks
WHERE name IN (%s)'''
sql = sql % in_p
print(sql)

stocks = pd.read_sql(sql, conmy)
stocks.set_index('name',inplace=True)
stocks

In [ ]:
#stocks.drop('SCC',inplace=True)
#stocks

In [ ]:
stocks.plot.bar()

In [ ]:
#stocks.reset_index('name',inplace=True)
#sns.barplot(x='name', y='price', data=stocks)

### Portfolio cumulative sum by period, name

In [ ]:
colw = 'period name shares unit_cost cost_amt mkt_amt cumsum'.split()
df['cumsum'] = df.groupby('period')['mkt_amt'].cumsum()
df[colw].sort_values(by=['period','name'],ascending=[True,True]).style.format(format_dict)